In [1]:
import requests
import pandas as pd

In [2]:
class YouTubeDataExtractor:
   
    def __init__(self, channel_id, api_key):
        self.channel_id = channel_id
        self.api_key = api_key
        self.myvideos = pd.DataFrame(columns=["Video Id","Video Title","Upload Date","View Count","Like Count","Comment Count"])
        self.column_header = ["Video Id","Video Title","Upload Date","View Count","Like Count","Comment Count"]
   
    def get_video(self, video):
        #Extract video ID, title, and upload date
        video_id = video["id"]["videoId"]
        video_title = video["snippet"]["title"]
        video_title = str(video_title).replace("&amp","")
        upload_date = video["snippet"]["publishedAt"]
        upload_date = str(upload_date).split("T")[0]
        return video_id, video_title, upload_date
       
    def get_videostats(self, video_id):
        #collect views, likes, dislikes, comment count
        url_videostats = f"https://www.googleapis.com/youtube/v3/videos?id={video_id}&part=statistics&key={self.api_key}"
        response_video_stats = requests.get(url_videostats).json()
        view_count = response_video_stats["items"][0]["statistics"]["viewCount"]
        like_count = response_video_stats["items"][0]["statistics"]["likeCount"]
        comment_count = response_video_stats["items"][0]["statistics"]["commentCount"]
        return view_count, like_count, comment_count
       
    def create_dataframe(self, video_id, video_title, upload_date, view_count, like_count, comment_count):
        #save video in pandas dataframe
        df = pd.DataFrame({"Video Id":[video_id],"Video Title":[video_title],"Upload Date":[upload_date],"View Count":[view_count],"Like Count":[like_count],"Comment Count":[comment_count]})
        df = df.set_axis(self.column_header, axis=1, inplace=False)
        frames = [self.myvideos, df]
        self.myvideos = pd.concat(frames, axis=0)
        #Convert string columns to the appropriate data types
        self.myvideos["Upload Date"] = pd.to_datetime(self.myvideos["Upload Date"])
        self.myvideos[["View Count", "Like Count", "Comment Count"]] = self.myvideos[["View Count", "Like Count", "Comment Count"]].astype(int)
        #Reset the index of the pandas dataframe
        self.myvideos = self.myvideos.reset_index().drop(columns="index")
        return self.myvideos
       
    def extract_data(self):
        #Build API request URL
        url_id = f"https://www.googleapis.com/youtube/v3/search?part=snippet&channelId={self.channel_id}&key={self.api_key}"
        #Send request to YouTube API and parse response
        response = requests.get(url_id).json()
        for video in response["items"]:
            if video["id"]["kind"]=="youtube#video":
                video_id, video_title, upload_date = self.get_video(video)
                view_count, like_count, comment_count = self.get_videostats(video_id)
                self.create_dataframe(video_id, video_title, upload_date, view_count, like_count, comment_count)
        return self.myvideos

 





In [3]:
youtubedata=YouTubeDataExtractor(channel_id,api_key)
youtubedata.extract_data()

,Video Id,Video Title,Upload Date,View Count,Like Count,Comment Count
0,U9cDSmDxFxo,LET YOUR FIRE FALL ON US |Piano| E flat - Jesu...,2020-07-27,28,6,0
1,RzFNaOIDPR8,FANTASY |Guitar| E major - Jesufemi Oresanya,2020-07-25,6,1,0
2,qFlRhZIaD2A,NEW DAWN | C major - Jesufemi Oresanya,2020-07-22,39,9,4
3,JbiNORm6XUA,GOOD TIME |Piano ; Guitar| B major - Jesufemi ...,2020-07-26,8,0,0
